# **INTRODUCTIONS**

---

Used for exceptional processes.

The code is separated into multiple sections:

1. **[VACCINE_COVERAGE_PER_AGE_GROUP_ARCHIVED_20220908](#VACCINE_COVERAGE_PER_AGE_GROUP_ARCHIVED_20220908)**
2. **[xxxx](#xxxx)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **VACCINE_COVERAGE_PER_AGE_GROUP_ARCHIVED_20220908**

---

table

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSARCHIVE].[CIMS_VACCINATED_PER_AGE_GROUP_GM_VR_ARCHIVED_20220908]'))
CREATE TABLE [VWSARCHIVE].[CIMS_VACCINATED_PER_AGE_GROUP_GM_VR_ARCHIVED_20220908](
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
  [DATE_OF_REPORT] [DATETIME] NULL,
  [DATE_OF_STATISTICS] [DATETIME] NULL,
  [REGION_CODE] [VARCHAR](10) NULL,
  [BIRTH_YEAR] [VARCHAR](100) NULL,
  [AGE_GROUP] [VARCHAR](10) NULL,    
  [VACCINATION_COVERAGE_ALL] [VARCHAR](100) NULL,  
  [VACCINATION_COVERAGE_ALL_LABEL] [VARCHAR](100) NULL,
  [VACCINATION_COVERAGE_COMPLETED] [VARCHAR](100) NULL,      
  [VACCINATION_COVERAGE_COMPLETED_LABEL] [VARCHAR](100) NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_DOF_DOS_RC_CIMS_VACCINATED_PER_AGE_GROUP_GM_VR] 
  ON [VWSARCHIVE].[CIMS_VACCINATED_PER_AGE_GROUP_GM_VR_ARCHIVED_20220908]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_DOF_DOS_RC_CIMS_VACCINATED_PER_AGE_GROUP_GM_VR] 
  ON [VWSARCHIVE].[CIMS_VACCINATED_PER_AGE_GROUP_GM_VR_ARCHIVED_20220908] (
    [DATE_LAST_INSERTED],
    [DATE_OF_REPORT],
    [DATE_OF_STATISTICS],
    [REGION_CODE]
  )  
  INCLUDE (
      [BIRTH_YEAR],
      [AGE_GROUP],
      [VACCINATION_COVERAGE_ALL],
      [VACCINATION_COVERAGE_ALL_LABEL],
      [VACCINATION_COVERAGE_COMPLETED],
      [VACCINATION_COVERAGE_COMPLETED_LABEL]
  );
GO

stored procedure

In [ ]:
-- Hiervoor eenmalig een query runnen die dit INTO een VWSARCHIVE tabel schrijft
-- met als extensie 20220908 en die filtert op de datum 2022-0908


-- 1) CREATE STORED PROCEDURE(S): ARCHIVE EXISTING DATA
CREATE OR ALTER PROCEDURE [DBO].[SP_CIMS_VACCINATED_AGE_GROUP_VR_GM_DEST_TO_ARCHIVE_20220908]
AS
BEGIN
    INSERT INTO [VWSARCHIVE].[CIMS_VACCINATED_PER_AGE_GROUP_GM_VR_ARCHIVED_20220908] (
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [REGION_CODE],
        [BIRTH_YEAR],
        [AGE_GROUP],    
        [VACCINATION_COVERAGE_ALL],  
        [VACCINATION_COVERAGE_ALL_LABEL],
        [VACCINATION_COVERAGE_COMPLETED],      
        [VACCINATION_COVERAGE_COMPLETED_LABEL],
        [DATE_LAST_INSERTED]
    )
    SELECT
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [REGION_CODE],
        [BIRTH_YEAR],
        [AGE_GROUP],    
        [VACCINATION_COVERAGE_ALL],  
        [VACCINATION_COVERAGE_ALL_LABEL],
        [VACCINATION_COVERAGE_COMPLETED],      
        [VACCINATION_COVERAGE_COMPLETED_LABEL],
        [DATE_LAST_INSERTED]
    FROM [VWSDEST].[CIMS_VACCINATED_PER_AGE_GROUP_GM_VR] WITH(NOLOCK)
    WHERE CAST([DATE_LAST_INSERTED] as Date) = CAST('2022-09-08' as Date)
END;
GO

VR Archive

View

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_CIMS_VACCINATED_PER_AGE_GROUP_VR_ARCHIVED_20220908] 
AS
    WITH CTE AS (
        SELECT
            dbo.CONVERT_DATETIME_TO_UNIX(DATE_OF_REPORT)                     AS DATE_OF_REPORT_UNIX,
            dbo.CONVERT_DATETIME_TO_UNIX(DATE_OF_STATISTICS)                 AS DATE_UNIX,
            [REGION_CODE]                                                    AS VRCODE,
            AGE_GROUP                                                        AS AGE_GROUP_RANGE,
            [BIRTH_YEAR]                                                     AS BIRTHYEAR_RANGE,
            CAST(ROUND(VACCINATION_COVERAGE_ALL, 0) AS INT)                  AS HAS_ONE_SHOT_PERCENTAGE,
            VACCINATION_COVERAGE_ALL_LABEL                                   AS HAS_ONE_SHOT_PERCENTAGE_LABEL,
            CAST(ROUND(VACCINATION_COVERAGE_COMPLETED, 0) AS INT)            AS FULLY_VACCINATED_PERCENTAGE,
            VACCINATION_COVERAGE_COMPLETED_LABEL                             AS FULLY_VACCINATED_PERCENTAGE_LABEL,
            dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED)                 AS DATE_OF_INSERTION_UNIX
        FROM VWSARCHIVE.CIMS_VACCINATED_PER_AGE_GROUP_GM_VR_ARCHIVED_20220908 WITH(NOLOCK)
        WHERE CAST([DATE_LAST_INSERTED] as Date) = CAST('2022-09-08' as Date)
            AND LEFT(REGION_CODE,2) = 'VR'
    )
    SELECT 
        [DATE_OF_REPORT_UNIX],
        T1.[DATE_UNIX],
        T1.[VRCODE],
        [AGE_GROUP_RANGE],
        [BIRTHYEAR_RANGE],
        [HAS_ONE_SHOT_PERCENTAGE],
        [HAS_ONE_SHOT_PERCENTAGE_LABEL],
        [FULLY_VACCINATED_PERCENTAGE],
        [FULLY_VACCINATED_PERCENTAGE_LABEL],
        T1.[DATE_OF_INSERTION_UNIX]
    FROM CTE T1
    WHERE [AGE_GROUP_RANGE] NOT IN ('12-17')
GO

protos

In [ ]:
-- 1) CREATE PROTOS CONFIGURATION(S).....
DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'VR_COLLECTION' AND NAME LIKE 'VR%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_CIMS_VACCINATED_PER_AGE_GROUP_VR_ARCHIVED_20220908',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_coverage_per_age_group_archived_20220908',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'VRCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'AGE_GROUP_RANGE|BIRTHYEAR_RANGE|FULLY_VACCINATED_PERCENTAGE|HAS_ONE_SHOT_PERCENTAGE|FULLY_VACCINATED_PERCENTAGE_LABEL|HAS_ONE_SHOT_PERCENTAGE_LABEL|DATE_OF_INSERTION_UNIX|DATE_UNIX',
        @layout_type_id INT = 2, -- VALUES
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO

GM ARCHIVE

View

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER   VIEW [VWSDEST].[V_CIMS_VACCINATED_PER_AGE_GROUP_GM_ARCHIVED_20220908]
AS   
    WITH CTE AS (
        SELECT
            dbo.CONVERT_DATETIME_TO_UNIX(DATE_OF_REPORT)                     AS DATE_OF_REPORT_UNIX,
            dbo.CONVERT_DATETIME_TO_UNIX(DATE_OF_STATISTICS)                 AS DATE_UNIX,
            [REGION_CODE]                                                    AS GMCODE,
            AGE_GROUP                                                        AS AGE_GROUP_RANGE,
            [BIRTH_YEAR]                                                     AS BIRTHYEAR_RANGE,
            CAST(ROUND(VACCINATION_COVERAGE_ALL, 0) AS INT)                  AS HAS_ONE_SHOT_PERCENTAGE,
            VACCINATION_COVERAGE_ALL_LABEL                                   AS HAS_ONE_SHOT_PERCENTAGE_LABEL,
            CAST(ROUND(VACCINATION_COVERAGE_COMPLETED, 0) AS INT)            AS FULLY_VACCINATED_PERCENTAGE,
            VACCINATION_COVERAGE_COMPLETED_LABEL                             AS FULLY_VACCINATED_PERCENTAGE_LABEL,
            dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED)                 AS DATE_OF_INSERTION_UNIX
        FROM [VWSARCHIVE].[CIMS_VACCINATED_PER_AGE_GROUP_GM_VR_ARCHIVED_20220908] WITH(NOLOCK)
        WHERE CAST([DATE_LAST_INSERTED] as Date) = CAST('2022-09-08' as Date)
            AND LEFT(REGION_CODE,2) = 'GM'
    )
    SELECT 
        [DATE_OF_REPORT_UNIX],
        T1.[DATE_UNIX],
        T1.[GMCODE],
        [AGE_GROUP_RANGE],
        [BIRTHYEAR_RANGE],
        [HAS_ONE_SHOT_PERCENTAGE],
        [HAS_ONE_SHOT_PERCENTAGE_LABEL],
        [FULLY_VACCINATED_PERCENTAGE],
        [FULLY_VACCINATED_PERCENTAGE_LABEL],
        T1.[DATE_OF_INSERTION_UNIX]
    FROM CTE T1
    WHERE [AGE_GROUP_RANGE] NOT IN ('12-17')
GO


protos

In [ ]:
-- 1) CREATE PROTOS CONFIGURATION(S).....
DECLARE @current_proto_name VARCHAR(50)

DECLARE @proto_name_table TABLE (
    [PROTO_NAME] VARCHAR(50)
);

-- 1) CREATE PROTO(S)...
INSERT INTO @proto_name_table 
SELECT Distinct [NAME]
    FROM [DATATINO_PROTO_1].[PROTOS]
    WHERE NAME NOT LIKE 'GM_COLLECTION' AND NAME LIKE 'GM%' 
ORDER BY NAME

DECLARE CUR CURSOR LOCAL FAST_FORWARD
FOR
SELECT [PROTO_NAME]
FROM @proto_name_table;

OPEN CUR;

FETCH NEXT FROM CUR
INTO @current_proto_name;

WHILE @@FETCH_STATUS = 0
BEGIN


-- 1) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_CIMS_VACCINATED_PER_AGE_GROUP_GM_ARCHIVED_20220908',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_coverage_per_age_group_archived_20220908',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = @current_proto_name,
        @constraint_key_name VARCHAR(50)= 'GMCODE',
        @grouped_key_name VARCHAR(50) = null,
        @grouped_last_update_name VARCHAR(50) = null,
        @proto_name VARCHAR(50) = @current_proto_name,
        @columns VARCHAR(256) = 'AGE_GROUP_RANGE|BIRTHYEAR_RANGE|FULLY_VACCINATED_PERCENTAGE|HAS_ONE_SHOT_PERCENTAGE|FULLY_VACCINATED_PERCENTAGE_LABEL|HAS_ONE_SHOT_PERCENTAGE_LABEL|DATE_OF_INSERTION_UNIX|DATE_UNIX',
        @layout_type_id INT = 2, -- VALUES
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

-- 2) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 3) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

FETCH NEXT FROM CUR
    INTO @current_proto_name;    
END

CLOSE CUR;
DEALLOCATE CUR;
DELETE FROM @proto_name_table;
GO